# All the parameters that need to be changed

In [1]:
# Missouri
state_ab = "mo"

## Data
1. Download all the data in directory "il_data"
2. Eextract them all

In [2]:
data_folder = state_ab + "_data/"
population1_data = "./{}{}_pl2020_b/{}_pl2020_p1_b.shp".format(data_folder, state_ab, state_ab)
population2_data = "./{}{}_pl2020_b/{}_pl2020_p2_b.shp".format(data_folder, state_ab, state_ab)
vap_data =  "./{}{}_pl2020_b/{}_pl2020_p4_b.shp".format(data_folder, state_ab, state_ab)
vest20_data = "./{}{}_vest_20/{}_vest_20.shp".format(data_folder, state_ab, state_ab)
vest18_data = "./{}{}_vest_18/{}_vest_18.shp".format(data_folder, state_ab, state_ab)
vest16_data = "./{}{}_vest_16/{}_vest_16.shp".format(data_folder, state_ab, state_ab)
cd_data = "./{}{}_cong_adopted_2022/5799H_02T.shp".format(data_folder, state_ab)
send_data = "./{}{}_sldu_adopted_2022/2022_JRC_Senate_Redistricting_Plan.shp".format(data_folder, state_ab)
hdist_data = "./{}{}_sldl_adopted_2022/HIBCC Final Apportionment Plan 2.shp".format(data_folder, state_ab)

## Parameters that needs to be manually checked

### base vest data
start_col = 5\
vest_base_data = vest20\
year = '20'

### district data
district column name of cong_df, send, hdist when calling add_dist()

# Program starts

In [3]:
import pandas as pd
import geopandas as gpd
import maup
import time
from maup import smart_repair
from gerrychain import Graph
import os

maup.progress.enabled = True

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
def do_smart_repair(df):
    # change it to the UTM it needs for smart_repair
    df = df.to_crs(df.estimate_utm_crs())
    df = smart_repair(df, disconnection_threshold=0, snap_precision=8)
    
    # check maup doctor again to see if smart repair works
    if maup.doctor(df) == True:
        # change it back to this UTM for this data
        df = df.to_crs('EPSG:4269')
    else:
        raise Exception('maup.doctor failed')
    
    return df

In [6]:
def add_district(dist_df, dist_name, election_df, col_name):
    # check if it needs to be smart_repair
    if maup.doctor(dist_df) != True:
        dist_df = do_smart_repair(dist_df)

    election_df = gpd.GeoDataFrame(election_df, crs="EPSG:4269")

    # assigne the pricincts
    precincts_to_district_assignment = maup.assign(election_df.geometry, dist_df.geometry)
    election_df[dist_name] = precincts_to_district_assignment
    for precinct_index in range(len(election_df)):
        election_df.at[precinct_index, dist_name] = dist_df.at[election_df.at[precinct_index, dist_name], col_name]

    return election_df

In [7]:
def rename(original, year):
    party = original[6]
    if party == 'R' or party == 'D':
        return original[3:6] + year + original[6]
    else:
        return original[3:6] + year + 'O'

In [8]:
def check_population(population, df):
    pop_check = pd.DataFrame({
        'pop_col': pop_col,
        'population_df': population[pop_col].sum(), 
        'vest_base': df[pop_col].sum(),
        'equal': [x == y for x, y in zip(population[pop_col].sum(), df[pop_col].sum())]
    })
    if pop_check['equal'].mean() < 1:
        print(pop_check)
        raise Exception("population doesn't agree")

    else:
        print("population agrees")

In [9]:
def add_vest(vest, df, year, population, start_col):    
     # check if it needs to be smart_repair
    if maup.doctor(vest) != True:
        vest = do_smart_repair(vest)
    
    # rename the columns
    original_col = vest.columns[start_col:-1]
    new_col = [rename(i, year) for i in original_col]
    rename_dict = dict(zip(original_col, new_col))
    vest = vest.rename(columns=rename_dict)
    vest = vest.groupby(level=0, axis=1).sum() # combine all the other party's vote into columns with sufix "O"
    col_name = list(set(new_col))
    col_name.sort()
    
    # make the blocks from precincts by weight
    vest = gpd.GeoDataFrame(vest, crs="EPSG:4269")
    election_in_block = population[["VAP", 'geometry']] # population_df is in block scale
    blocks_to_precincts_assignment = maup.assign(election_in_block.geometry, vest.geometry)
    weights = election_in_block["VAP"] / blocks_to_precincts_assignment.map(election_in_block["VAP"].groupby(blocks_to_precincts_assignment).sum())
    weights = weights.fillna(0)
    prorated = maup.prorate(blocks_to_precincts_assignment, vest[col_name], weights)
    election_in_block[col_name] = prorated
    
    # assign blocks to precincts
    election_in_block = gpd.GeoDataFrame(election_in_block, crs="EPSG:4269")
    df = gpd.GeoDataFrame(df, crs="EPSG:4269")
    block_to_pricinct_assginment = maup.assign(election_in_block.geometry, df.geometry)
    df[col_name] = election_in_block[col_name].groupby(block_to_pricinct_assginment).sum()
    df = df.groupby(level=0, axis=1).sum()
    
    # check if population agrees
    check_population(population, df)
    
    return df

## Read the census data

In [10]:
population1_df = gpd.read_file(population1_data)
population2_df = gpd.read_file(population2_data)
vap_df = gpd.read_file(vap_data)

In [11]:
population2_df = population2_df.drop(columns=['SUMLEV', 'LOGRECNO', 'GEOID', 'COUNTY', 'geometry'])
vap_df = vap_df.drop(columns=['SUMLEV', 'LOGRECNO', 'GEOID', 'COUNTY', 'geometry'])

In [12]:
population_df = pd.merge(population1_df, population2_df, on='GEOID20')
population_df = pd.merge(population_df, vap_df, on='GEOID20')

In [13]:
rename_dict = {'P0020001': 'TOTPOP', 'P0020002': 'HISP', 'P0020005': 'NH_WHITE', 'P0020006': 'NH_BLACK', 'P0020007': 'NH_AMIN',
               'P0020008': 'NH_ASIAN', 'P0020009': 'NH_NHPI', 'P0020010': 'NH_OTHER', 'P0020011': 'NH_2MORE',
               'P0040001': 'VAP', 'P0040002': 'HVAP', 'P0040005': 'WVAP', 'P0040006': 'BVAP', 'P0040007': 'AMINVAP',
               'P0040008': 'ASIANVAP', 'P0040009': 'NHPIVAP', 'P0040010': 'OTHERVAP', 'P0040011': '2MOREVAP'}

In [14]:
population_df.rename(columns=rename_dict, inplace = True)

In [15]:
population_df['H_WHITE'] = population_df.apply(lambda t: t['P0010003'] - t['NH_WHITE'], 1)
population_df['H_BLACK'] = population_df.apply(lambda t: t['P0010004'] - t['NH_BLACK'], 1)
population_df['H_AMIN'] = population_df.apply(lambda t: t['P0010005'] - t['NH_AMIN'], 1)
population_df['H_ASIAN'] = population_df.apply(lambda t: t['P0010006'] - t['NH_ASIAN'], 1)
population_df['H_NHPI'] = population_df.apply(lambda t: t['P0010007'] - t['NH_NHPI'], 1)
population_df['H_OTHER'] = population_df.apply(lambda t: t['P0010008'] - t['NH_OTHER'], 1)
population_df['H_2MORE'] = population_df.apply(lambda t: t['P0010009'] - t['NH_2MORE'], 1)

# Read the base vest data
Now using it as a "base precinct", but it could be vest 18 or vest 16 if vest 20 is not working

In [16]:
def add_vest_base(vest, start_col, year):
    original_col = vest.columns[start_col:-1]
    new_col = [rename(i, year) for i in original_col]
    rename_dict = dict(zip(original_col, new_col))
    vest = vest.rename(columns=rename_dict)
    vest = vest.groupby(level=0, axis=1).sum()
    vest = gpd.GeoDataFrame(vest, crs="EPSG:4269")
    
    return vest

### Check if vest 20 can be used as base

In [17]:
vest20 = gpd.read_file(vest20_data)

In [18]:
if maup.doctor(vest20) != True:
    vest20 = do_smart_repair(vest20)

100%|████████████████████████████████████████| 3733/3733 [00:10<00:00, 365.08it/s]


There are 19 overlaps.
There are 1 holes.
Snapping all geometries to a grid with precision 10^( -3 ) to avoid GEOS errors.
Identifying overlaps...


100%|███████████████████████████████████████| 4308/4308 [00:02<00:00, 1564.02it/s]


Resolving overlaps...
Assigning order 2 pieces...
Filling gaps...


100%|████████████████████████████████████████| 3733/3733 [00:09<00:00, 385.39it/s]


### If it is true for maup doctor, we will use it as the base vest data.
Check where the election column starts, this should be the same for all vest data in that state

In [19]:
vest20.columns

Index(['STATEFP', 'COUNTYFP', 'NAME', 'G20PRERTRU', 'G20PREDBID', 'G20PRELJOR',
       'G20PREGHAW', 'G20PRECBLA', 'G20GOVRPAR', 'G20GOVDGAL', 'G20GOVLCOM',
       'G20GOVGBAU', 'G20LTGRKEH', 'G20LTGDCAN', 'G20LTGLSLA', 'G20LTGGDRA',
       'G20ATGRSCH', 'G20ATGDFIN', 'G20ATGLBAB', 'G20SOSRASH', 'G20SOSDFAL',
       'G20SOSLFRE', 'G20SOSGLEH', 'G20SOSCVEN', 'G20TRERFIT', 'G20TREDENG',
       'G20TRELKAS', 'G20TREGCIV', 'geometry'],
      dtype='object')

## Parameters that need to be checked

In [34]:
start_col = 3
vest_base_data = vest20
year = '20'

In [23]:
vest_base = add_vest_base(vest_base_data, start_col, year)

In [24]:
# vap and population have the same GEOID20
blocks_to_precincts_assignment = maup.assign(population_df.geometry, vest_base.geometry)

100%|█████████████████████████████████████████| 3733/3733 [01:22<00:00, 45.03it/s]


In [25]:
pop_col = ['TOTPOP', 'HISP', 'NH_WHITE', 'NH_BLACK', 'NH_AMIN', 'NH_ASIAN', 'NH_NHPI', 'NH_OTHER', 'NH_2MORE', 'H_WHITE', 'H_BLACK', 'H_AMIN', 'H_ASIAN', 'H_NHPI', 'H_OTHER', 'H_2MORE', 'VAP', 'HVAP', 'WVAP', 'BVAP', 'AMINVAP', 'ASIANVAP', 'NHPIVAP', 'OTHERVAP', '2MOREVAP']

In [26]:
vest_base[pop_col] = population_df[pop_col].groupby(blocks_to_precincts_assignment).sum()

In [27]:
election_df = gpd.GeoDataFrame(vest_base, crs="EPSG:4269")

### Check if the population agrees

In [28]:
check_population(population_df, vest_base)

population agrees


# Add more vest data

In [29]:
vest18 = gpd.read_file(vest18_data)

In [30]:
vest18.columns

Index(['STATEFP', 'COUNTYFP', 'NAME', 'G18USSRHAW', 'G18USSDMCC', 'G18USSLCAM',
       'G18USSGCRA', 'G18USSIODE', 'G18AUDRMCD', 'G18AUDDGAL', 'G18AUDLOTO',
       'G18AUDGFIT', 'G18AUDCLUE', 'geometry'],
      dtype='object')

In [31]:
# check the result here
start_col = 3
election_df = add_vest(vest18, election_df, '18', population_df, start_col)

100%|████████████████████████████████████████| 3274/3274 [00:09<00:00, 328.35it/s]


There are 8 overlaps.
Snapping all geometries to a grid with precision 10^( -3 ) to avoid GEOS errors.
Identifying overlaps...


100%|███████████████████████████████████████| 3970/3970 [00:02<00:00, 1496.68it/s]


Resolving overlaps...
Assigning order 2 pieces...
Filling gaps...


Gaps to simplify: 0it [00:00, ?it/s]
Gaps to fill: 0it [00:00, ?it/s]
100%|█████████████████████████████████████████| 3733/3733 [01:37<00:00, 38.32it/s]


population agrees


### vest 16

In [33]:
vest16 = gpd.read_file(vest16_data)
vest16.columns

Index(['STATEFP', 'COUNTYFP', 'NAME', 'G16PRERTRU', 'G16PREDCLI', 'G16PRELJOH',
       'G16PREGSTE', 'G16PRECCAS', 'G16USSRBLU', 'G16USSDKAN', 'G16USSLDIN',
       'G16USSGMCF', 'G16USSCRYM', 'G16GOVRGRE', 'G16GOVDKOS', 'G16GOVLSPR',
       'G16GOVGFIT', 'G16GOVITUR', 'G16LTGRPAR', 'G16LTGDCAR', 'G16LTGLHED',
       'G16LTGGLEA', 'G16ATGRHAW', 'G16ATGDHEN', 'G16TRERSCH', 'G16TREDBAK',
       'G16TRELOTO', 'G16TREGHEX', 'G16SOSRASH', 'G16SOSDSMI', 'G16SOSLMOR',
       'geometry'],
      dtype='object')

In [35]:
start_col = 3
election_df = add_vest(vest16, election_df, '16', population_df, start_col)

100%|████████████████████████████████████████| 3324/3324 [00:09<00:00, 358.31it/s]


There are 83 overlaps.
Snapping all geometries to a grid with precision 10^( -3 ) to avoid GEOS errors.
Identifying overlaps...


100%|███████████████████████████████████████| 4109/4109 [00:02<00:00, 1448.37it/s]


Resolving overlaps...
Assigning order 2 pieces...
Assigning order 3 pieces...
Filling gaps...


Gaps to simplify: 0it [00:00, ?it/s]
Gaps to fill: 0it [00:00, ?it/s]
100%|█████████████████████████████████████████| 3733/3733 [01:48<00:00, 34.45it/s]


population agrees


## Add the district data

In [36]:
cong_df = gpd.read_file(cd_data).to_crs('EPSG:4269')
send = gpd.read_file(send_data).to_crs('EPSG:4269')
hdist = gpd.read_file(hdist_data).to_crs('EPSG:4269')

In [37]:
cong_df.head()

,ID,AREA,DISTRICT,MEMBERS,LOCKED,NAME,POPULATION,IDEAL_VALU,DEVIATION,F_DEVIATIO,DISTRICT_L,geometry
0,1,258.333252,1,1.0,None,None,769364,769364.0,0.0,0.0,1|0%,"POLYGON ((-90.42195 38.67338, -90.42198 38.673..."
1,2,1819.885010,2,1.0,None,None,769364,769364.0,0.0,0.0,2|0%,"POLYGON ((-90.68129 38.76518, -90.68133 38.764..."
2,3,7692.436030,3,1.0,None,None,769364,769364.0,0.0,0.0,3|0%,"POLYGON ((-90.66318 38.93422, -90.66285 38.933..."
3,4,14654.767500,4,1.0,None,None,769364,769364.0,0.0,0.0,4|0%,"POLYGON ((-92.69589 38.22203, -92.69739 38.222..."
4,5,431.047668,5,1.0,None,None,769364,769364.0,0.0,0.0,5|0%,"POLYGON ((-94.27817 38.83506, -94.28759 38.835..."


In [38]:
send.head()

,ID,AREA,DISTRICT,MEMBERS,LOCKED,NAME,POPULATION,IDEAL_VALU,DEVIATION,F_DEVIATIO,...,F_PERFINDE,PERFINDEX1,F_PERFIND1,PERFINDEX2,PERFINDEX3,F_PERFIND2,PERFINDEX4,F_PERFIND3,PERFINDEX5,geometry
0,2,80.03,SD14,1.0,None,None,185861,181027.0,4834.0,0.0267,...,0.7887,160258.5738,0.2113,758581.3127,376688.6996,0.7817,105199.5418,0.2183,481888.2414,"POLYGON ((-90.28099 38.71773, -90.28127 38.717..."
1,4,83.23,SD13,1.0,None,None,185863,181027.0,4836.0,0.0267,...,0.8166,140598.1802,0.1834,766789.8417,393529.0964,0.8131,90441.8049,0.1869,483970.9013,"POLYGON ((-90.28278 38.71029, -90.28314 38.710..."
2,8,45.07,SD5,1.0,None,None,180730,181027.0,-297.0,-0.0016,...,0.9115,57341.6639,0.0885,647598.4425,363370.7868,0.9035,38809.0459,0.0965,402179.8327,"POLYGON ((-90.16627 38.77273, -90.16636 38.772..."
3,10,34.31,SD4,1.0,None,None,184241,181027.0,3214.0,0.0178,...,0.7292,217849.0733,0.2708,804505.1674,395167.0106,0.7315,145053.8345,0.2685,540220.8451,"POLYGON ((-90.23947 38.60462, -90.23852 38.604..."
4,12,59.94,SD1,1.0,None,None,184853,181027.0,3826.0,0.0211,...,0.5449,374550.2798,0.4551,823017.8952,301769.0488,0.5374,259731.8251,0.4626,561500.8739,"POLYGON ((-90.38260 38.49098, -90.38257 38.491..."


In [39]:
hdist.head()

,ID,AREA,DISTRICT,MEMBERS,LOCKED,NAME,POPULATION,IDEAL_VALU,DEVIATION,F_DEVIATIO,DISTRICT_L,Dist_Num,Class,geometry
0,1,2387.352050,1,1.0,None,None,36931,37760.0,-829.0,-0.021954,1|-2.2%,1,1,"POLYGON ((-94.63203 40.57137, -94.63219 40.565..."
1,2,23.070000,10,1.0,None,None,37451,37760.0,-309.0,-0.008200,10|-0.82%,10,2,"POLYGON ((-94.87656 39.82088, -94.87511 39.820..."
2,3,11.047083,100,1.0,None,None,37955,37760.0,195.0,0.005164,100|0.52%,100,1,"POLYGON ((-90.55845 38.59270, -90.55862 38.592..."
3,4,26.420000,101,1.0,None,None,37597,37760.0,-163.0,-0.004300,101|-0.43%,101,2,"POLYGON ((-90.67860 38.66646, -90.67844 38.666..."
4,5,201.246994,102,1.0,None,None,36867,37760.0,-893.0,-0.023649,102|-2.36%,102,1,"POLYGON ((-90.63510 38.75664, -90.63391 38.756..."


In [40]:
election_df = add_district(cong_df, "CD", election_df, "DISTRICT")

100%|███████████████████████████████████████████████| 8/8 [00:07<00:00,  1.05it/s]


In [41]:
election_df = add_district(send, "SEND", election_df, "DISTRICT")

100%|█████████████████████████████████████████████| 34/34 [00:07<00:00,  4.36it/s]


In [42]:
election_df = add_district(hdist, "HDIST", election_df, "DISTRICT")

100%|███████████████████████████████████████████| 163/163 [00:11<00:00, 13.84it/s]


In [43]:
list(election_df.columns)

['2MOREVAP',
 'AMINVAP',
 'ASIANVAP',
 'ATG16D',
 'ATG16R',
 'ATG20D',
 'ATG20O',
 'ATG20R',
 'AUD18D',
 'AUD18O',
 'AUD18R',
 'BVAP',
 'COUNTYFP',
 'GOV16D',
 'GOV16O',
 'GOV16R',
 'GOV20D',
 'GOV20O',
 'GOV20R',
 'HISP',
 'HVAP',
 'H_2MORE',
 'H_AMIN',
 'H_ASIAN',
 'H_BLACK',
 'H_NHPI',
 'H_OTHER',
 'H_WHITE',
 'LTG16D',
 'LTG16O',
 'LTG16R',
 'LTG20D',
 'LTG20O',
 'LTG20R',
 'NAME',
 'NHPIVAP',
 'NH_2MORE',
 'NH_AMIN',
 'NH_ASIAN',
 'NH_BLACK',
 'NH_NHPI',
 'NH_OTHER',
 'NH_WHITE',
 'OTHERVAP',
 'PRE16D',
 'PRE16O',
 'PRE16R',
 'PRE20D',
 'PRE20O',
 'PRE20R',
 'SOS16D',
 'SOS16O',
 'SOS16R',
 'SOS20D',
 'SOS20O',
 'SOS20R',
 'STATEFP',
 'TOTPOP',
 'TRE16D',
 'TRE16O',
 'TRE16R',
 'TRE20D',
 'TRE20O',
 'TRE20R',
 'USS16D',
 'USS16O',
 'USS16R',
 'USS18D',
 'USS18O',
 'USS18R',
 'VAP',
 'WVAP',
 'geometry',
 'CD',
 'SEND',
 'HDIST']

### Put the base precinct year after the precinct information column

In [44]:
base_columns = {}
if 'COUNTYFP' + year not in election_df.columns:
    base_columns = {
        'STATEFP':'STATEFP'+year,
        'COUNTYFP':'COUNTYFP'+year,
        'PRECINCT':'PRECINCT'+year,
        'GEOID':'GEOID'+year,
        'NAME':'NAME'+year}
election_df.rename(columns=base_columns, inplace = True)

In [46]:
# reorder the columns
fixed_columns = [
    'STATEFP'+year,
    'COUNTYFP'+year,
    # 'PRECINCT'+year,
    # 'GEOID'+year,
    'NAME'+year,
    'CD',
    'SEND',
    'HDIST',
    'TOTPOP',
    'NH_2MORE',
    'NH_AMIN',
    'NH_ASIAN',
    'NH_BLACK',
    'NH_NHPI',
    'NH_OTHER',
    'NH_WHITE',
    'HISP',
    'H_AMIN',
    'H_ASIAN',
    'H_BLACK',
    'H_NHPI',
    'H_OTHER',
    'H_WHITE',
    'H_2MORE',
    'VAP',
    'HVAP',
    'WVAP',
    'BVAP',
    'AMINVAP',
    'ASIANVAP',
    'NHPIVAP',
    'OTHERVAP',
    '2MOREVAP']

election_columns = [col for col in election_df.columns if col not in fixed_columns]
final_col = fixed_columns + election_columns
election_df = election_df[final_col]

In [47]:
list(election_df.columns)

['STATEFP20',
 'COUNTYFP20',
 'NAME20',
 'CD',
 'SEND',
 'HDIST',
 'TOTPOP',
 'NH_2MORE',
 'NH_AMIN',
 'NH_ASIAN',
 'NH_BLACK',
 'NH_NHPI',
 'NH_OTHER',
 'NH_WHITE',
 'HISP',
 'H_AMIN',
 'H_ASIAN',
 'H_BLACK',
 'H_NHPI',
 'H_OTHER',
 'H_WHITE',
 'H_2MORE',
 'VAP',
 'HVAP',
 'WVAP',
 'BVAP',
 'AMINVAP',
 'ASIANVAP',
 'NHPIVAP',
 'OTHERVAP',
 '2MOREVAP',
 'ATG16D',
 'ATG16R',
 'ATG20D',
 'ATG20O',
 'ATG20R',
 'AUD18D',
 'AUD18O',
 'AUD18R',
 'GOV16D',
 'GOV16O',
 'GOV16R',
 'GOV20D',
 'GOV20O',
 'GOV20R',
 'LTG16D',
 'LTG16O',
 'LTG16R',
 'LTG20D',
 'LTG20O',
 'LTG20R',
 'PRE16D',
 'PRE16O',
 'PRE16R',
 'PRE20D',
 'PRE20O',
 'PRE20R',
 'SOS16D',
 'SOS16O',
 'SOS16R',
 'SOS20D',
 'SOS20O',
 'SOS20R',
 'TRE16D',
 'TRE16O',
 'TRE16R',
 'TRE20D',
 'TRE20O',
 'TRE20R',
 'USS16D',
 'USS16O',
 'USS16R',
 'USS18D',
 'USS18O',
 'USS18R',
 'geometry']

In [48]:
# store the result in directory "il"
os.makedirs("./{}".format(state_ab))
election_df.to_file("./{}/{}.shp".format(state_ab, state_ab))
election_df.to_file('./{}/{}.geojson'.format(state_ab, state_ab), driver='GeoJSON')

# Only do once to build json and read from file when generating ensembles
graph = Graph.from_file("./{}/{}.shp".format(state_ab, state_ab), ignore_errors=True)
graph.to_json("./{}/{}.json".format(state_ab, state_ab))